In [3]:
import numpy as np
from sklearn.datasets.samples_generator import make_blobs
base_x, base_y = make_blobs(n_samples=22000, centers=3, n_features=3, random_state=0, center_box=(10, 20))

#随机生成3类数据
base_n = len(base_y)
base_y=base_y.reshape((base_n,1))

base_x-=base_x.min()
base_x/=base_x.max()

train_x = base_x[:20000]
train_y = base_y[:20000]
val_x = base_x[20000:]
val_y = base_y[20000:]

print(train_x.shape,train_y.shape,val_x.shape,val_y.shape)

(20000, 3) (20000, 1) (2000, 3) (2000, 1)


In [37]:
#单模型
#1. rf
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

rf = RandomForestClassifier(max_depth=4, random_state=0)
rf.fit(train_x, train_y)

y_pre = rf.predict(val_x)
c = 0.0
for i in range(2000):
    if y_pre[i] == val_y[i]:
        c+=1.0
print(c/2000.0)

#2. gbdt
from sklearn.ensemble import GradientBoostingRegressor
gbdt=GradientBoostingRegressor(loss='ls', learning_rate=0.1, n_estimators=100, max_depth=3)
gbdt.fit(train_x, train_y)

y_pre=gbdt.predict(val_x)

def v2num(v):
    np.abs(v-2)
y_num = []
for yy in y_pre:
    y_num.append(round(yy,0))
c = 0.0
for i in range(2000):
    if y_num[i] == val_y[i]:
        c+=1.0
print(c/2000.0)


#3.

f:\software\python\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys
f:\software\python\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.933
0.9275


In [24]:
# stacking
from sklearn.cross_validation import KFold
class Ensemble(object):
    def __init__(self, n_folds, stacker, base_models):
        self.n_folds = n_folds            #交叉验证集划分的折数
        self.stacker = stacker            #第二层stacking时使用的分类器
        self.base_models = base_models    #第一层的基本模型 们
    def fit_predict(self, X, y, T):
        X = np.array(X)   #train_x
        y = np.array(y)   #train_y
        T = np.array(T)   #test_x
        folds = list(KFold(len(y), n_folds=self.n_folds, shuffle=True, random_state=2016))
        #sklearn.cross_validation.KFold(n, n_folds=3, shuffle=False, random_state=None)
        #http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.KFold.html
        #这里只是生成了index 的迭代器，根据index取数据在后面进行
        
        S_train = np.zeros((X.shape[0], len(self.base_models))) #第二层的训练数据
        S_test = np.zeros((T.shape[0], len(self.base_models)))
        #数据条数不变，特征数变为模型数，因为每个模型产生一列
        
        for i, clf in enumerate(self.base_models):    #clf  Classification  
            S_test_i = np.zeros((T.shape[0], len(folds)))
            for j, (train_idx, test_idx) in enumerate(folds):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]
                # y_holdout = y[test_idx]
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_holdout)[:]
                S_train[test_idx, i] = y_pred
                S_test_i[:, j] = clf.predict(T)[:]  #整个T的预测
            S_test[:, i] = S_test_i.mean(1)    #按行求平均值 即axis=1. 矩阵变成一列后加入S_test中
            
        self.stacker.fit(S_train, y)
        y_pred = self.stacker.predict(S_test)[:]
        return y_pred

f:\software\python\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [36]:
rfS = RandomForestClassifier(max_depth=4, random_state=0)
gbdtS =GradientBoostingRegressor(loss='ls', learning_rate=0.1, n_estimators=100, max_depth=3)
modelS = Ensemble(5, rfS, [rfS, gbdtS])
preS = modelS.fit_predict(train_x,train_y,val_x)
#print(preS)
c = 0.0
for i in range(2000):
    if preS[i] == val_y[i]:
        c+=1.0
print(c/2000.0)

f:\software\python\lib\site-packages\ipykernel_launcher.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
f:\software\python\lib\site-packages\ipykernel_launcher.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
f:\software\python\lib\site-packages\ipykernel_launcher.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
f:\software\python\lib\site-packages\ipykernel_launcher.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
f:\software\python\lib\site-packages\ipykernel_launcher.py:28: DataConversionWarning: A column-vector y was passed when a 1d arr

0.9335


f:\software\python\lib\site-packages\ipykernel_launcher.py:34: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
